In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader

In [2]:
torch.manual_seed(1)

In [8]:
class CustomDataset(Dataset):
    def __init__(self):
        self.x_data = [[1, 2], 
                       [2, 3], 
                       [3, 1], 
                       [4, 3], 
                       [5, 3], 
                       [6, 2]]
        
        self.y_data = [[0], [0], [0], [1], [1], [1]]

    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self, index):
        x_train = torch.FloatTensor(self.x_data[index])
        y_train = torch.FloatTensor(self.y_data[index])

        return x_train, y_train


In [12]:
class LogisticRegression(nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.linear(x)
        x = self.sigmoid(x)
        return x


In [14]:
dataset = CustomDataset()
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

print(len(dataset))

for idx, samples in enumerate(dataset):
    print("Index : {} /Sample : {}".format(idx+1, samples))
    
model = LogisticRegression()

optimizer = optim.SGD(model.parameters(), lr=1)


6
Index : 1 /Sample : (tensor([1., 2.]), tensor([0.]))
Index : 2 /Sample : (tensor([2., 3.]), tensor([0.]))
Index : 3 /Sample : (tensor([3., 1.]), tensor([0.]))
Index : 4 /Sample : (tensor([4., 3.]), tensor([1.]))
Index : 5 /Sample : (tensor([5., 3.]), tensor([1.]))
Index : 6 /Sample : (tensor([6., 2.]), tensor([1.]))


In [16]:
nb_epochs = 100
for epoch in range(nb_epochs + 1):
    for batch_idx, samples in enumerate(dataloader):
        x_train, y_train = samples
        # H(x) 계산
        hypothesis = model(x_train)

        # cost 계산
        cost = F.binary_cross_entropy(hypothesis, y_train)

        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        # 20번마다 로그 출력
        if epoch % 10 == 0:
            prediction = hypothesis >= torch.FloatTensor([0.5]) # 예측값이 0.5를 넘으면 True로 간주
            correct_prediction = prediction.float() == y_train # 실제값과 일치하는 경우만 True로 간주
            accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
            print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format( # 각 에포크마다 정확도를 출력
                epoch, nb_epochs, cost.item(), accuracy * 100,
            ))

Epoch    0/100 Cost: 0.291346 Accuracy 100.00%
Epoch    0/100 Cost: 0.000080 Accuracy 100.00%
Epoch    0/100 Cost: 0.000504 Accuracy 100.00%
Epoch    0/100 Cost: 3.923525 Accuracy 0.00%
Epoch    0/100 Cost: 0.001345 Accuracy 100.00%
Epoch    0/100 Cost: 0.023497 Accuracy 100.00%
Epoch   10/100 Cost: 4.169055 Accuracy 0.00%
Epoch   10/100 Cost: 10.327231 Accuracy 0.00%
Epoch   10/100 Cost: 0.583346 Accuracy 100.00%
Epoch   10/100 Cost: 2.094786 Accuracy 0.00%
Epoch   10/100 Cost: 2.321112 Accuracy 0.00%
Epoch   10/100 Cost: 0.000000 Accuracy 100.00%
Epoch   20/100 Cost: 0.000031 Accuracy 100.00%
Epoch   20/100 Cost: 0.000000 Accuracy 100.00%
Epoch   20/100 Cost: 0.321043 Accuracy 100.00%
Epoch   20/100 Cost: 0.000104 Accuracy 100.00%
Epoch   20/100 Cost: 0.018242 Accuracy 100.00%
Epoch   20/100 Cost: 4.428900 Accuracy 0.00%
Epoch   30/100 Cost: 0.000000 Accuracy 100.00%
Epoch   30/100 Cost: 0.000000 Accuracy 100.00%
Epoch   30/100 Cost: 0.000419 Accuracy 100.00%
Epoch   30/100 Cost: 8.3

In [17]:
print(list(model.parameters()))

[Parameter containing:
tensor([[4.3716, 2.0195]], requires_grad=True), Parameter containing:
tensor([-19.1765], requires_grad=True)]
